#### Language Translation using Sequence to Sequence Learning in Keras with Encoders and Decoders

In [1]:
# Blog link - https://www.youtube.com/redirect?q=https%3A%2F%2Fblog.keras.io%2Fa-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html&v=f-JCCOHwx1c&redir_token=QUFFLUhqbDYydnd2WkZHSHhZZEd6ZklJNzBfMXJHc2g5UXxBQ3Jtc0tuVzFNSlQxVkgxVFpkdXA2OG5VQVZmNUFWcmpGQUV3TmdkQ1AxZ21RT3J6YTZ2Ung4djBSZnhDQXlQci05UF8taGNuRFF6WnVsMFlROXEyV2xDZ3NIRndTaTB5QlFtRElwWW4wMXE3UEsxRlhuRnp2Zw%3D%3D&event=video_description
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [2]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = 'fra-eng/fra.txt'

In [3]:
input_texts = []
target_texts = []
with open(data_path, 'r', encoding = 'utf-8') as f:
    lines = f.read().split('\n')
for line in lines[:min(num_samples, len(lines) -1)]:
    input_text, target_text, *_ = line.split('\t')
    target_text = '\t'+ target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

In [4]:
input_characters = sorted(list(set("".join(input_texts))))
target_characters = sorted(list(set("".join(target_texts))))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max(list(map(len, input_texts)))
max_decoder_seq_length = max(list(map(len, target_texts)))

In [5]:
print('Number of samples: ', len(input_texts))
print('Number of unique input tokens: ', num_encoder_tokens)
print('Number of unique target tokens: ', num_decoder_tokens)
print('Max sequence length for inputs: ', max_encoder_seq_length)
print('Max sequence length for target: ', max_decoder_seq_length)

Number of samples:  10000
Number of unique input tokens:  71
Number of unique target tokens:  94
Max sequence length for inputs:  15
Max sequence length for target:  59


In [6]:
input_token_index = (dict(enumerate(input_characters)))
input_token_index = dict(zip(input_token_index.values(), input_token_index.keys()))
target_token_index = (dict(enumerate(target_characters)))
target_token_index = dict(zip(target_token_index.values(), target_token_index.keys()))

In [7]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype = 'float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')

In [8]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1
    
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1
    decoder_input_data[i, t+1: , target_token_index[' ']] = 1
    decoder_target_data[i, t:, target_token_index[' ']] = 1

#### Creating an Encoder Decoder model

In [9]:
# Define an input sequence and process it
encoder_inputs = Input(shape = (None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard 'encoder_outputs' and only keep the states.
encoder_states = [state_h, state_c]

In [10]:
# Set up the decoder, using 'encoder_states' as initial state
decoder_inputs = Input(shape = (None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we eill use them in inference

decoder_lstm = LSTM(latent_dim, return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [11]:
# Define the model that will turn
# 'encoder_input_data' & 'decoder_input_data' into 'decoder_target_data'
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)

Epoch 1/100
125/125 [==============================] - 39s 312ms/step - loss: 1.1511 - accuracy: 0.7278 - val_loss: 1.0288 - val_accuracy: 0.7234
Epoch 2/100
125/125 [==============================] - 34s 276ms/step - loss: 0.8365 - accuracy: 0.7741 - val_loss: 0.8064 - val_accuracy: 0.7743
Epoch 3/100
125/125 [==============================] - 29s 234ms/step - loss: 0.6713 - accuracy: 0.8107 - val_loss: 0.7184 - val_accuracy: 0.7939
Epoch 4/100
125/125 [==============================] - 40s 322ms/step - loss: 0.5905 - accuracy: 0.8293 - val_loss: 0.6610 - val_accuracy: 0.8058
Epoch 5/100
125/125 [==============================] - 53s 420ms/step - loss: 0.5406 - accuracy: 0.8424 - val_loss: 0.6118 - val_accuracy: 0.8185
Epoch 6/100
125/125 [==============================] - 55s 439ms/step - loss: 0.5042 - accuracy: 0.8521 - val_loss: 0.5934 - val_accuracy: 0.8241
Epoch 7/100
125/125 [==============================] - 52s 414ms/step - loss: 0.4756 - accuracy: 0.8599 - val_loss: 0.5637 -

##### For inference,
1) Encode the input sentence and retrieve the initial decoder state 

2) Run one step of the decoder with this initial state and a "start of sequence" token as target. The output will be the next target character.

3) Append the target character predicted and repeat.

In [12]:
# Here is our inference setup
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape = (latent_dim, ))
decoder_state_input_c = Input(shape = (latent_dim, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [13]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [18]:
decode_sequence(encoder_input_data[0].reshape([1,encoder_input_data[0].shape[0], encoder_input_data[0].shape[1]]))

NameError: name 'reverse_target_char_index' is not defined

In [16]:
encoder_input_data[0].shape

(15, 71)